In [15]:
import numpy as np
import regex as re
import os
import math
import pandas as pd 
import sys
from openpyxl import load_workbook, Workbook
from openpyxl.utils.cell import get_column_letter

path_src = "/local/scratch/Figaro/figaro-code"
exp_names = [ "figaro_lapack", "figaro_thin", "mkl"]
figaro_impls = ["figaro_lapack", "figaro_thin"]
exp_paths = {"figaro_lapack": "dumps/figaro/lapack", "mkl": "dumps/python/mkl", "figaro_thin": "dumps/figaro/thin_diag"}
csv_name = "R.csv"

row_nums = [512]
column_nums =[2]
row_nums = [512, 1024, 2048, 4096]
column_nums = [1, 4, 16, 64, 256, 1024]
row_col_nums = []
np.set_printoptions(threshold=sys.maxsize, precision=14)
join_order = "FullJoin"

for row_num in row_nums:
    for col_num in column_nums:
        if (row_num > col_num):
            row_col_nums.append((row_num, col_num))

row_col_nums = row_col_nums[:21]

def arrange_up_triang(n: int):
    A  = np.zeros( (n, n))
    idx = 1
    for row_idx in range(n):
        for col_idx in range(row_idx, n):
            A[row_idx, col_idx] = idx
            idx += 1
    return A


def compare_rs(R: np.array, num_rows: int, num_cols: int):
    R_expX = arrange_up_triang(num_cols) * math.sqrt(num_rows)
    diff = R[:num_cols, :num_cols] - R_expX
    #print(R[:num_cols, :num_cols])
    #print(R_expX)
    #print(diff)
    rel_fro = np.linalg.norm(diff, ord='fro') / np.linalg.norm(R_expX, ord='fro')
    return rel_fro

exp_rel_errors = {}

for exp_name in exp_names:
    #Initialize paths for the current experiment 
    exp_path = exp_paths[exp_name]
    accur_path = os.path.join(path_src, exp_path)
    df_measurement = pd.DataFrame(columns=column_nums)
    dump_r_path = os.path.join(accur_path, csv_name)


    for db_idx, db_num in enumerate(range(1, len(row_col_nums) + 1)): 
        db_name = "DBCartesianProductAccuracy{}".format(db_num)
        path_csv = os.path.join(accur_path, db_name, join_order, csv_name)
        
        # Writng header files
        row_num = row_col_nums[db_idx][0]
        col_num = row_col_nums[db_idx][1]
        r_mat = np.genfromtxt(path_csv, delimiter=',')
        rel_error = compare_rs(r_mat, row_num, col_num)
        df_measurement.at[row_num, col_num] = rel_error
        df_measurement = df_measurement.astype(float)
        df_measurement.to_csv(exp_name+"comp.csv")
    exp_rel_errors[exp_name] = df_measurement
    



In [22]:
def df_to_latex(measurement: pd.DataFrame):
    df_measurement_lat = measurement.rename(index={512:" $2^9$", 1024: "$2^{10}$", 
        2048: "$2^{11}$", 4096: "$2^{12}$", 8192: "$2^{13}$"}, 
        columns={64:"2^6", 128:"$2^7$", 256: "$2^8$", 512: "$2^9$", 1024:"$2^{10}$", 4096: "$2^{12}$"})
    return df_measurement_lat

for exp_name in exp_names:
    df_measurement = exp_rel_errors[exp_name]
    
    print(exp_name)
    df_measur_lat = df_to_latex(df_measurement)
    print(df_measur_lat.to_latex(float_format="%.2g",escape=False, na_rep=" "))
for figaro_impl in figaro_impls:
    df_error = exp_rel_errors["mkl"] / exp_rel_errors[figaro_impl]
    df_error_lat = df_to_latex(df_error)
    print(figaro_impl)
    print(df_error_lat.to_latex(float_format="%.2g",escape=False, na_rep=" "))


figaro_lapack
\begin{tabular}{lrrrrrr}
\toprule
{} &       1 &       4 &      16 &     2^6 &   $2^8$ &  $2^{10}$ \\
\midrule
 $2^9$   & 9.4e-16 & 1.5e-15 & 1.7e-15 & 2.5e-15 & 9.7e-15 &           \\
$2^{10}$ &       0 & 1.6e-16 & 9.1e-16 & 3.1e-15 & 1.3e-14 &           \\
$2^{11}$ & 9.4e-16 & 9.8e-16 & 1.2e-15 & 3.1e-15 & 1.9e-14 &     7e-14 \\
$2^{12}$ &       0 &       0 &   7e-16 & 3.7e-15 & 2.2e-14 &           \\
\bottomrule
\end{tabular}

figaro_thin
\begin{tabular}{lrrrrrr}
\toprule
{} &       1 &       4 &      16 &     2^6 &   $2^8$ &  $2^{10}$ \\
\midrule
 $2^9$   & 9.4e-16 & 1.5e-15 & 1.8e-15 & 3.1e-15 & 1.4e-14 &           \\
$2^{10}$ &       0 & 1.6e-16 & 1.4e-15 & 3.5e-15 & 1.7e-14 &           \\
$2^{11}$ & 9.4e-16 & 1.3e-15 & 1.3e-15 & 3.3e-15 & 2.5e-14 &   1.1e-13 \\
$2^{12}$ &       0 & 1.6e-15 & 1.7e-15 & 4.2e-15 & 2.7e-14 &           \\
\bottomrule
\end{tabular}

mkl
\begin{tabular}{lrrrrrr}
\toprule
{} &       1 &       4 &      16 &     2^6 &   $2^8$ &  $2^{10}$ \\
